In [53]:
# TEMP - Tesing DUMP
x = np.array([1,-2,.3,-.1])
x = tf.keras.backend.constant(x)
A = tf.constant(tf.ones(np.array([4,4])))
b = tf.constant(tf.ones(4))

# NEU Unit (Reconfiguration Map)

## Initialize Modules

###### Imports

In [66]:
# Deep Learning & ML
import tensorflow as tf
import keras as kr
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Layer
from keras import utils as np_utils

# General
import numpy as np
import time

# Message Boxes
import tkinter
from tkinter import messagebox

## Build Custom Layers.  

These will be combined to form the NEU unit.  

### Shift Layer: x -> x+c ; c in R

In [67]:
class BiasLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(BiasLayer, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.bias = self.add_weight('bias',
                                    shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
    def call(self, x):
        return x + self.bias

#### Diagonal/Scaling Layer: x -> Diag(k_1,...,k_n)* x ; k_i in R

Rescaling is a special case of multiplication by a diagonal matrix.  This can be imposed as a weight constraint.

In [68]:
class DiagonalWeight(Constraint):
    """Constrains the weights to be diagonal.
    """
    def __call__(self, w):
        N = K.int_shape(w)[-1]
        m = K.eye(N)
        w *= m
        return w

NameError: name 'Constraint' is not defined

### Activation Function Applying Bump Function Element-wise

In [69]:
def bump_activation(x_in):
    x_in_abs = tf.math.abs(x)
    x_out = tf.where(tf.math.abs(x) >= 1, 0 , tf.pow(1-x,-1))
    return x_out

1. First applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.

2. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)
                    \mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}$
3. Adds trainable $d\times d$ (weight) matrix $A$ and trainable bias (vector) $b \in \mathbb{R}^d$ to that output:

In [78]:
class bump_function_layer(keras.layers.Layer):

    def __init__(self, dim):
        super(Custom_layer, self).__init__()
        self.dim = dim

        # add trainable weight
        self.weight = self.add_weight(shape=(dim,dim),trainable=True)
        # add trainable bias
        self.bias = self.add_weight(shape=(dim))

    def call(self, input):
        # 1) Bump Function
        x_in_abs = tf.math.abs(x)
        x_thresheld = tf.where(tf.math.abs(x) >= 1, 0 , tf.math.exp(tf.pow(1-x,-1)))
        # 2) Map to Diagonal Matrix
        x_out = tf.expand_dims(x_thresheld, 1)
        # 3) Apply weights and biases
        x_out = (x_out * self.weight) + self.bias
        # 4) Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        # Return Output
        return x_out
        
    def get_config(self):
        config = super(bump_function_layer, self).get_config()
        config['dim'] = self.dim
        return config

NameError: name 'keras' is not defined

#### Custom Layer: x -> diag(x) -> Wdiag(x); W trainable affine map from d -> d

In [6]:
# This code is to hide the main tkinter window
root = tkinter.Tk()
root.withdraw

# Message Box
messagebox.showinfo("Note", "The map x->W(x) need not be invertiable since everything works out after applying exponential map (right now were working in the tangent space of GL_d)")

'ok'

In [8]:
tf.keras.layers.Lambda(
    tf.linalg.diag
)

In [72]:
class Custom_layer(keras.layers.Layer):

    def __init__(self, dim):
        super(Custom_layer, self).__init__()
        self.dim = dim

        # add trainable weight
        self.weight = self.add_weight(shape=(dim,dim),trainable=True)
        # add trainable bias
        self.bias = self.add_weight(shape=(dim))

    def call(self, input):
        # your function
        return (tf.linalg.diag(input)*self.weight) + self.bias
    def get_config(self):
        config = super(Custom_layer, self).get_config()
        config['dim'] = self.dim
        return config

NameError: name 'keras' is not defined